In [ ]:

# Loading libraries
import seaborn as sns
import pandas as pd
import csv
import warnings
import plotly.express as px
warnings.filterwarnings("ignore")

In [ ]:
# EV Energy: Wh/km tahmini - 3 model karşılaştırma (lightgbm ilk etapta olabilir)
!pip -q install lightgbm shap scikit-learn pandas numpy matplotlib

In [ ]:
import pandas as pd
import numpy as np

csv_path = "/content/final_training_data.csv"
df = pd.read_csv(csv_path)

print("Şekil:", df.shape)
df.head(3)


Şekil: (167210, 25)


,timestamp,vehicle_id,speed_kmh,lat,lon,z,acceleration,mass_kg,soc_pc,energy_consumption,...,recuperationEfficiency,airDragCoefficient,rollDragCoefficient,frontSurfaceArea,constantPowerIntake,rotatingMass,propulsionEfficiency,maximumPower,dist_m,slope_pct
0,2,veh1,0.000,39.749175,30.508940,822.0,0.00,1958.0,96.554181,0.00,...,0.94,0.344,0.01,2.47,484.0,39.0,0.94,193684.0,NaN,NaN
1,3,veh1,14.112,39.749195,30.508903,822.0,3.92,1958.0,96.549121,4.90,...,0.94,0.344,0.01,2.47,484.0,39.0,0.94,193684.0,3.914897,0.0
2,4,veh1,28.224,39.749237,30.508829,822.0,3.92,1958.0,96.534407,14.25,...,0.94,0.344,0.01,2.47,484.0,39.0,0.94,193684.0,7.829794,0.0


In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    Provides the names of categorical, numeric, and categorical but cardinal variables in the dataset.
    Note: Numeric-looking categorical variables are also included in categorical variables.

    Parameters
    ------
        dataframe: dataframe
                The dataframe from which variable names are to be taken
        cat_th: int, optional
                Class threshold value for numeric but categorical variables
        car_th: int, optinal
                Class threshold value for categorical but cardinal variables

    Returns
    ------
        cat_cols: list
                List of categorical variables
        num_cols: list
                List of numeric variables
        cat_but_car: list
                List of categorical-looking cardinal variables

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))

    Notes
    ------
        cat_cols + num_cols + cat_but_car = total number of variables
        num_but_cat is within cat_cols.
        The sum of the 3 lists returned is equal to the total number of variables:
        cat_cols + num_cols + cat_but_car = number of variables
    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car, num_but_cat


cat_cols, num_cols, cat_but_car, num_but_cat = grab_col_names(df) # Let's use the 'grab_col_names' function to examine the categories of our variables and classify them.
print("cat cols: ", cat_cols)
print("num_cols: ", num_cols)
print("cat_but_car: ", cat_but_car)
print("num_but_cat" ,num_but_cat)


Observations: 167210
Variables: 25
cat_cols: 1
num_cols: 23
cat_but_car: 1
num_but_cat: 1
cat cols:  ['rollDragCoefficient']
num_cols:  ['timestamp', 'speed_kmh', 'lat', 'lon', 'z', 'acceleration', 'mass_kg', 'soc_pc', 'energy_consumption', 'accel', 'decel', 'radialDragCoefficient', 'device.battery.capacity', 'device.battery.maximumChargeRate', 'recuperationEfficiency', 'airDragCoefficient', 'frontSurfaceArea', 'constantPowerIntake', 'rotatingMass', 'propulsionEfficiency', 'maximumPower', 'dist_m', 'slope_pct']
cat_but_car:  ['vehicle_id']
num_but_cat ['rollDragCoefficient']


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167210 entries, 0 to 167209
Data columns (total 25 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   timestamp                         167210 non-null  int64  
 1   vehicle_id                        167210 non-null  object 
 2   speed_kmh                         167210 non-null  float64
 3   lat                               167210 non-null  float64
 4   lon                               167210 non-null  float64
 5   z                                 167210 non-null  float64
 6   acceleration                      167210 non-null  float64
 7   mass_kg                           167210 non-null  float64
 8   soc_pc                            167210 non-null  float64
 9   energy_consumption                167210 non-null  float64
 10  accel                             167210 non-null  float64
 11  decel                             167210 non-null  f

In [ ]:
# sayısal olması gereken sütunları sayıya çevir
# grab_col_names çıktısında rollDragCoefficient num_but_cat görünüyor (unique<10). Bu bir sayısal katsayı; kategorik kodlanmamalı.

#vehicle_id yüksek kardinal kimlik → özellik olarak verilmez, sadece gruplama için kullanılır.

NUM_SHOULD_BE = [
    "timestamp","speed_kmh","lat","lon","z","acceleration","mass_kg","soc_pc",
    "energy_consumption","accel","decel","radialDragCoefficient","device.battery.capacity",
    "device.battery.maximumChargeRate","recuperationEfficiency","airDragCoefficient",
    "rollDragCoefficient","frontSurfaceArea","constantPowerIntake","rotatingMass",
    "propulsionEfficiency","maximumPower","dist_m","slope_pct"
]
for c in NUM_SHOULD_BE:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# id sütunu dursun ama X'e girmeyecek
if "vehicle_id" not in df.columns:
    df["vehicle_id"] = "veh0"

İVME ÇOK ÖNEMLİ

In [ ]:
# ivme ayrımı
# bu sayede enerji tüketimi (hızlanırken) ve rejeneratif enerji kazanımı (yavaşlarken) ayrı ayrı analiz edilebilir.
# verimlilik hesaplarını ve optimizasyonu daha doğru yapmayı sağlar
# Pozitif ivme (gaz) tüketimi artırır; negatif ivme (fren/iniş) rejenerasyon sağlayabilir. Tek sütun olsa bu fark kaybolabilirdi
df["acc_pos"] = df["acceleration"].clip(lower=0)
df["acc_neg"] = (-df["acceleration"]).clip(lower=0)

In [ ]:
# Hız: m/s ve v^2
df["speed_ms"] = df["speed_kmh"] / 3.6
df["v2"] = df["speed_ms"] ** 2

# Aerodinamik: Cd * A
df["CdA"] = df["airDragCoefficient"] * df["frontSurfaceArea"] # bu ikini birleştirip vermek daha mantıklı

# Çok küçük mesafelerde Wh/km şişmesin
df.loc[df["dist_m"] < 0.5, "dist_m"] = np.nan # Çok küçük mesafe olursa oran fırlar. Bu satırları dışarıda bırakmak, hedef gürültüsünü temizler.
df["target_Wh_per_km"] = df["energy_consumption"] / (df["dist_m"] / 1000.0)
df = df.dropna(subset=["target_Wh_per_km"]).reset_index(drop=True)


In [ ]:
df.head()

,timestamp,vehicle_id,speed_kmh,lat,lon,z,acceleration,mass_kg,soc_pc,energy_consumption,...,propulsionEfficiency,maximumPower,dist_m,slope_pct,acc_pos,acc_neg,speed_ms,v2,CdA,target_Wh_per_km
0,3,veh1,14.11200,39.749195,30.508903,822.0,3.9200,1958.0,96.549121,4.90,...,0.94,193684.0,3.914897,0.000000,3.9200,0.000,3.9200,15.366400,0.84968,1251.629365
1,4,veh1,28.22400,39.749237,30.508829,822.0,3.9200,1958.0,96.534407,14.25,...,0.94,193684.0,7.829794,0.000000,3.9200,0.000,7.8400,61.465600,0.84968,1819.971324
2,5,veh1,18.96840,39.749265,30.508779,822.0,-2.5710,1958.0,96.543039,-8.37,...,0.94,193684.0,5.262141,0.000000,0.0000,2.571,5.2690,27.762361,0.84968,-1590.607470
3,6,veh1,24.00264,39.749316,30.508752,821.0,1.3984,1958.0,96.537380,5.48,...,0.94,193684.0,6.136851,-16.295001,1.3984,0.000,6.6674,44.454223,0.84968,892.966075
4,7,veh1,24.00264,39.749363,30.508801,821.0,0.0000,1958.0,96.535532,1.80,...,0.94,193684.0,6.652306,0.000000,0.0000,-0.000,6.6674,44.454223,0.84968,270.582866


In [ ]:
df.shape

(138983, 31)

In [ ]:
# diğer sayısal parametreler
for c in ["mass_kg","rollDragCoefficient","propulsionEfficiency","recuperationEfficiency",
          "constantPowerIntake","maximumPower","device.battery.maximumChargeRate"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

feature_cols = [
    "speed_ms","v2","acceleration","acc_pos","acc_neg",
    #"slope_pct", yokuş etkisini nasıl vermeliyiz modele ?
    "mass_kg","CdA","rollDragCoefficient",
    "propulsionEfficiency","recuperationEfficiency",
    "constantPowerIntake","maximumPower","device.battery.maximumChargeRate",
]
feature_cols = [c for c in feature_cols if c in df.columns]
X = df[feature_cols].copy()
y = df["target_Wh_per_km"].copy()
print("X shape:", X.shape)

X shape: (138983, 13)


In [ ]:
# aynı araca ait satırların train/val/test’e karışmamasını sağlar. Yani vehicle_id bazlı ayırır; veri sızıntısını (leakage) önler.
from sklearn.model_selection import GroupShuffleSplit

if "vehicle_id" not in df.columns:
    df["vehicle_id"] = "veh0"
groups = df["vehicle_id"]

# GroupShuffleSplit ile önce test setini ayırıyor. Bu aşamada bir araca ait tüm satırlar ya testte ya da train+val tarafında kalır; asla bölünmez.
def group_split(X, y, groups, test_size=0.15, val_size=0.15, rs=42):
    """
    X, y ve groups'u alır; önce TEST setini, sonra kalan içinden VALIDATION setini ayırır.
    NOT: Aynı 'vehicle_id'ye ait TÜM satırlar ya train/val tarafında kalır ya da testte olur;
         asla bir araca ait satırlar birden fazla parçaya bölünmez.
    """
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=rs)
    i_tr, i_te = next(gss.split(X, y, groups))
    gss2 = GroupShuffleSplit(n_splits=1, test_size=val_size/(1-test_size), random_state=rs)
    j_tr, j_val = next(gss2.split(X.iloc[i_tr], y.iloc[i_tr], groups.iloc[i_tr]))
    tr_idx = X.iloc[i_tr].index[j_tr]; val_idx = X.iloc[i_tr].index[j_val]; te_idx = X.index[i_te]
    return tr_idx, val_idx, te_idx

train_idx, val_idx, test_idx = group_split(X, y, groups)
X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_val,   y_val   = X.loc[val_idx],   y.loc[val_idx]
X_test,  y_test  = X.loc[test_idx],  y.loc[test_idx]
print(f"Split -> train:{len(train_idx)}, val:{len(val_idx)}, test:{len(test_idx)}")


Split -> train:97567, val:20646, test:20770


In [ ]:
# --- vehicle_id -> split eşlemesi ve listeler ---

split = pd.Series(index=X.index, data="train")
split.loc[val_idx] = "val"
split.loc[test_idx] = "test"

# 1) Her vehicle_id tek bir split'te mi? (güvenlik kontrolü)
tmp = df.loc[split.index, ["vehicle_id"]].assign(split=split.values)
assert (tmp.groupby("vehicle_id")["split"].nunique() == 1).all(), "Aynı vehicle_id birden fazla split'te!"

# 2) vehicle_id -> split haritası
veh_to_split = tmp.groupby("vehicle_id")["split"].first().reset_index()
print(veh_to_split["split"].value_counts(), "\n")
print(veh_to_split.head())

# 3) Listeler (train/val/test) ve kısa özet
veh_train = veh_to_split.loc[veh_to_split["split"]=="train","vehicle_id"].tolist()
veh_val   = veh_to_split.loc[veh_to_split["split"]=="val","vehicle_id"].tolist()
veh_test  = veh_to_split.loc[veh_to_split["split"]=="test","vehicle_id"].tolist()

print(f"\n#vehicle_id -> train:{len(veh_train)}, val:{len(veh_val)}, test:{len(veh_test)}")
print("train örnek:", veh_train)
print("val   örnek:", veh_val)
print("test  örnek:", veh_test)

# 4) Araç başına satır sayısı (bilgi amaçlı)
rows_per_vehicle = tmp.groupby(["vehicle_id","split"]).size().reset_index(name="rows")
print("\nAraç başına satır (ilk 10):")
print(rows_per_vehicle.head(10))


rows_per_vehicle.to_csv("/content/rows_per_vehicle.csv", index=False)
print("\nKaydedildi:")
print("/content/rows_per_vehicle.csv")


split
train    1049
val       226
test      225
Name: count, dtype: int64 

  vehicle_id  split
0       veh1  train
1      veh10  train
2     veh100  train
3    veh1000  train
4    veh1001  train

#vehicle_id -> train:1049, val:226, test:225
train örnek: ['veh1', 'veh10', 'veh100', 'veh1000', 'veh1001', 'veh1002', 'veh1003', 'veh1004', 'veh1005', 'veh1006', 'veh1008', 'veh1009', 'veh101', 'veh1010', 'veh1012', 'veh1013', 'veh1014', 'veh1015', 'veh1016', 'veh1017', 'veh1018', 'veh102', 'veh1021', 'veh1022', 'veh1023', 'veh1025', 'veh1026', 'veh1029', 'veh1031', 'veh1032', 'veh1033', 'veh1034', 'veh1035', 'veh1036', 'veh1039', 'veh104', 'veh1040', 'veh1041', 'veh1046', 'veh1047', 'veh1048', 'veh1049', 'veh1050', 'veh1053', 'veh1054', 'veh1058', 'veh1060', 'veh1063', 'veh1064', 'veh1065', 'veh1066', 'veh107', 'veh1070', 'veh1071', 'veh1072', 'veh1073', 'veh1074', 'veh1076', 'veh1077', 'veh1078', 'veh1079', 'veh108', 'veh1081', 'veh1083', 'veh1084', 'veh1086', 'veh1087', 'veh1088', 'veh108